# Procesamiento del habla
Primero, vamos a importar los paquetes necesarios para el desarrollo.

In [ ]:
%matplotlib inline
import os
import plotly
import wav

import tensorflow as tf
import numpy as np

from matplotlib import pyplot
from IPython.display import Audio

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.utils import shuffle
from sklearn.base import BaseEstimator

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Convolution1D, MaxPooling1D, Dropout, Flatten

from plotly.graph_objs import Scatter, Layout, Bar, Figure

plotly.offline.init_notebook_mode()

### VERSION 1: RECONOCIMIENTO UTILIZANDO ÚNICAMENTE LA ONDA

Enunciado del problema: dados 184 audios clasificados, se pide crear un clasificador que detecte si el hablante es de sexo mujer o hombre.
Analisis del problema: el audio mas corto posee 318764 frames. Es decir, estaríamos manejando una matriz de X con dimension (184, 318764). En general, para crear un clasificador que generalice bien necesitamos que m >> n donde (m, n). Esto quiere decir, que tengamos muchas mas instancias que cantidad de parametros. Lo que suele suceder al tener pocas instancias con muchos parámetros es que en vez de generalizar, terminamos produciendo mucho overfitting (nos pegamos mucho a las enormes cantidades de parametros y tenemos muy pocas muestras distintas).

Un primer approach va a ser aumentar nuestro set de datos. El audio mas corto dura 19 segundos. Lo que vamos a hacer es a cada audio cortarlo en pedazos de 4 segundos cada uno. Esto viene de que lo que nos importa es capturar los patrones en la voz que nos permitan definir si es hombre o mujer, no nos interesa la secuencia de palabras que dice.

Primero vamos a definir nuestor set de entrenamiento y testeo y luego extenderlos.

Comencemos primero definiendo algunas variables globales (parámetros del enunciado).

In [ ]:
RUTA_DIRECTORIO_DATOS = "datos"
VENTANA_EN_SEGUNDOS = 4
SAMPLE_RATE = 16000
MINIMA_CANTIDAD_DE_FRAMES_A_PROCESAR = 318764 # Es el minimo de todos en este caso
CANTIDAD_DE_FRAMES_A_PROCESAR = VENTANA_EN_SEGUNDOS * SAMPLE_RATE

Ahora vamos a cargar los archivos IPU y WAV

In [ ]:
archivos_en_carpeta_datos = os.listdir(RUTA_DIRECTORIO_DATOS)
archivos_wav = []
archivos_ipu = []
for archivo in archivos_en_carpeta_datos:
    if (archivo[-3:] == "ipu"):
        archivos_ipu.append(archivo)
    elif (archivo[-3:] == "wav"):
        archivos_wav.append(archivo)

Creo mi set de datos (X's)

In [ ]:
X = []
leng = []
for archivo_wav in archivos_wav:
    data, frames, _, duration = wav.load_from_wav(RUTA_DIRECTORIO_DATOS + "/" + archivo_wav)
    leng.append(len(data))
    X.append(data[:MINIMA_CANTIDAD_DE_FRAMES_A_PROCESAR])

X = np.asarray(X)
print(X.shape)

Creo las etiquetas (mis Y's)

In [ ]:
y = []
y_sgd = []
for archivo_wav in archivos_wav:
    if archivo_wav[3] == "m":
        y.append(1)
    elif archivo_wav[3] == "f":
        y.append(0)
y = np.asarray(y)

print(y.shape)

Testeo que los datos hayan sido cargados correctamente

In [ ]:
Audio(data=X[25], rate=SAMPLE_RATE)

In [ ]:
print(y[25])

Separo en un set de entrenamiento y testing:

In [ ]:
X, y = shuffle(X, y, random_state=42)
X_train = X[:150]
X_test = X[150:180]
y_train = y[:150]
y_test = y[150:180]

print("Shape X_train {}".format(np.shape(X_train)))
print("Shape X_test {}".format(np.shape(X_test)))
print("Shape y_train {}".format(np.shape(y_train)))
print("Shape y_test {}".format(np.shape(y_test)))

Agrando los set de datos para evitar overfitting:

In [ ]:
def augument_data(dataset, labels):
    augumented_dataset = []
    augumented_labels = []

    for data, label in zip(dataset, labels):
        lower_bound = 0
        upper_bound = CANTIDAD_DE_FRAMES_A_PROCESAR
        augumented_data = []
        # corto en 4 pedazos el audio
        for i in range(4):
            augumented_data.append(data[lower_bound:upper_bound])
            augumented_labels.append(label)
            lower_bound = upper_bound
            upper_bound += CANTIDAD_DE_FRAMES_A_PROCESAR
        augumented_dataset.extend(augumented_data)

    augumented_dataset = np.asarray(augumented_dataset)
    augumented_labels = np.asarray(augumented_labels)
    return augumented_dataset, augumented_labels

In [ ]:
X_train, y_train = augument_data(X_train, y_train)
X_test, y_test = augument_data(X_test, y_test)

In [ ]:
print("Shape X_train {}".format(np.shape(X_train)))
print("Shape X_test {}".format(np.shape(X_test)))
print("Shape y_train {}".format(np.shape(y_train)))
print("Shape y_test {}".format(np.shape(y_test)))

Creamos nuestro perceptron multicapa

In [ ]:
def mlp_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(4, input_dim=64, kernel_initializer=init, activation='relu'))
    model.add(Dense(2, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

Ahora definimos nuestra red neuronal convolucional

In [ ]:
def conv_model(optimizer='rmsprop', init='glorot_uniform'):
    # CREO MIS CONVOLUCIONES
    model = Sequential()
    model.add(Convolution1D(16, 5, strides=1, padding='same', kernel_initializer=init, activation='relu', 
                            input_shape=(64000, 1)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Convolution1D(32, 5, strides=1, padding='same', kernel_initializer=init, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Convolution1D(64, 5, strides=1, padding='same', kernel_initializer=init, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Convolution1D(128, 5, strides=1, padding='same', kernel_initializer=init, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Convolution1D(256, 5, strides=1, padding='same', kernel_initializer=init, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))    
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512, kernel_initializer=init, activation='relu'))
    model.add(Dense(256, kernel_initializer=init, activation='relu'))
    model.add(Dense(128, kernel_initializer=init, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

Ahora el siguiente paso es definir nuestro preprocesador de la entrada y nuestro clasificador. Para esta tarea vamos a utilizar Pipelines que se encargan de enviar mensajes de manera secuencial a los objetos que definamos en el Pipeline. En este caso vamos a pedirle el preproceso al objeto encargado de calcular las componentes principales, las cuales van a alimentar a nuestro clasificador.

In [ ]:
class Reshape(BaseEstimator):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
#mlp_model = KerasClassifier(build_fn=mlp_model, verbose=0)
conv_model = KerasClassifier(build_fn=conv_model, verbose=0)

estimators = [("reshaper", Reshape()), ('clf', conv_model)]
pipeline = Pipeline(estimators)

Vamos a utilizar GridSearch para probar distintas configuraciones de parámetros sobre PCA y la red neuronal.

In [ ]:
OPTIMIZERS = ['adam']
EPOCHS = [10]
BATCHES = [10]
INIT = ['glorot_uniform']

Creamos nuestro diccionario en el formato que toma GridSearchCV para efectivamente ejecutar las diferentes configuraciones

In [ ]:
param_grid = [
    {
        'clf__optimizer': OPTIMIZERS,
        'clf__epochs': EPOCHS,
        'clf__batch_size': BATCHES,
        'clf__init': INIT
    },
]

Para ver que también generaliza nuestro clasificador vamos a utilizar KFold Cross-Validation, en particular con 10 folds.

In [ ]:
kfold = KFold(n_splits=5, shuffle=True)

Ahora nos falta fittear nuestro algoritmo y calcular los resultados sobre el conjunto de testeo

In [ ]:
grid_search = GridSearchCV(estimator=pipeline, cv=kfold, param_grid=param_grid)
grid_result = grid_search.fit(X_train, y_train)
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
best_params = grid_search.best_params_
best_n_components = best_params['reduce_dim__n_components']
print("La mejor mejor configuración de parámetros es: \n" + "Cantidad de componentes principales: " + str(best_n_components))

Una vez realizado esto, creamos los gráficos. Para esto creamos dos métodos, uno para la configuración del gráfico (es decir, necesitamos parsear los resultados obtenidos por PCA y KNN de manera que PlotLy los entienda y pueda graficar) y finalmente un método que nos permita graficar usando PlotLy.

In [ ]:
def graph_config(legends, results):
    old_n_neighbors = legends[0]['knn__n_neighbors']
    set_of_lists_with_results = []
    list_with_results = []
    graph_names = []
    for index, legend in enumerate(legends):
        new_n_neighbors = legend['knn__n_neighbors']

        if new_n_neighbors == old_n_neighbors:
            list_with_results.append(results[index])
        else:
            graph_names.append(old_n_neighbors)
            set_of_lists_with_results.append(list_with_results)
            list_with_results = [results[index]]
            old_n_neighbors = new_n_neighbors
    graph_names.append(new_n_neighbors)
    set_of_lists_with_results.append(list_with_results)
    return graph_names, set_of_lists_with_results

def graph(N_COMPONENTS, graph_names, set_of_lists_with_results):
    traces = []
    for index, set in enumerate(set_of_lists_with_results):
        x = N_COMPONENTS
        y = set
        name = "cantidad de vecinos = " + str(graph_names[index])
        traces.append(Scatter(x=x, y=y, name=name))
    layout = Layout(
        xaxis=dict(
            title='Cantidad de componentes principales',
            type='log',
            autorange=True
        ),
        yaxis=dict(
            title='% Accuracy',
            type='log',
            autorange=True
        ),
        title="Medida de performance - Accuracy"
    )
    figure = Figure(data=traces, layout=layout)
    plotly.offline.iplot(figure)

Una vez hecho esto, pasemos a graficar:

In [ ]:
#graph_names, set_of_lists_with_results = graph_config(legends, results)
graph(N_COMPONENTS, ["legend"], [results])